In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Read the feature file into a DataFrame
df = pd.read_csv(feature_files[0])

list_of_column_names = list(df.columns.drop(['Arousal_Value', 'Valence_Value', 'frameTime']))

print(list_of_column_names)
print(len(list_of_column_names))

['F0final_sma_stddev', 'F0final_sma_amean', 'voicingFinalUnclipped_sma_stddev', 'voicingFinalUnclipped_sma_amean', 'jitterLocal_sma_stddev', 'jitterLocal_sma_amean', 'jitterDDP_sma_stddev', 'jitterDDP_sma_amean', 'shimmerLocal_sma_stddev', 'shimmerLocal_sma_amean', 'logHNR_sma_stddev', 'logHNR_sma_amean', 'audspec_lengthL1norm_sma_stddev', 'audspec_lengthL1norm_sma_amean', 'audspecRasta_lengthL1norm_sma_stddev', 'audspecRasta_lengthL1norm_sma_amean', 'pcm_RMSenergy_sma_stddev', 'pcm_RMSenergy_sma_amean', 'pcm_zcr_sma_stddev', 'pcm_zcr_sma_amean', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma[0]_amean', 'audSpec_Rfilt_sma[1]_stddev', 'audSpec_Rfilt_sma[1]_amean', 'audSpec_Rfilt_sma[2]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[3]_stddev', 'audSpec_Rfilt_sma[3]_amean', 'audSpec_Rfilt_sma[4]_stddev', 'audSpec_Rfilt_sma[4]_amean', 'audSpec_Rfilt_sma[5]_stddev', 'audSpec_Rfilt_sma[5]_amean', 'audSpec_Rfilt_sma[6]_stddev', 'audSpec_Rfilt_sma[6]_amean', 'audSpec_Rfilt_sma[7

In [3]:
! pip install opensmile
import pandas as pd
import opensmile

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

# Process the audio file
y = smile.process_file('/content/drive/MyDrive/FYP_dataset/CNN Features/abc.wav')

# Set Pandas options to display more rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the extracted features
smile_features= set((y.keys()))
print(smile_features)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168842 sha256=8c920467950f0daaff55bb7ad542d09d0f5471c0bc17e7e05bb88395ed32c094
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639
{'audSpec_Rfilt_sma[6]_peakRangeRel', 'pcm_zcr_sma_segLenStddev', 'audSpec_Rfilt_sma_de[14]_risetime', 'audSpec_Rfilt_sma_de[5]_iqr2-3', 'audSpec_Rfilt_sma_de[9]_peakRangeAbs', 'mfcc_sma_de[7]

In [4]:
set_our_features=set(list_of_column_names)
print(set_our_features)

{'audSpec_Rfilt_sma_de[16]_stddev', 'pcm_fftMag_spectralRollOff90.0_sma_de_stddev', 'pcm_fftMag_mfcc_sma_de[9]_stddev', 'audSpec_Rfilt_sma[10]_amean', 'audSpec_Rfilt_sma[25]_amean', 'audSpec_Rfilt_sma_de[15]_stddev', 'audSpec_Rfilt_sma[17]_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'audSpec_Rfilt_sma_de[21]_amean', 'pcm_fftMag_spectralSlope_sma_amean', 'shimmerLocal_sma_stddev', 'audSpec_Rfilt_sma_de[23]_stddev', 'pcm_fftMag_mfcc_sma_de[10]_stddev', 'F0final_sma_amean', 'audspecRasta_lengthL1norm_sma_de_amean', 'audSpec_Rfilt_sma_de[24]_amean', 'audSpec_Rfilt_sma[24]_stddev', 'audSpec_Rfilt_sma[19]_amean', 'jitterLocal_sma_de_stddev', 'shimmerLocal_sma_de_amean', 'audSpec_Rfilt_sma_de[7]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[17]_amean', 'audSpec_Rfilt_sma_de[10]_stddev', 'audSpec_Rfilt_sma[12]_stddev', 'audSpec_Rfilt_sma[3]_stddev', 'pcm_fftMag_spectralSkewness_sma_amean', 'jitterDDP_sma_stddev', 'pcm_RMSenergy_sma_stddev', 'audSpec_Rfilt_sma_de[6]_stddev', 'pcm_fft

In [5]:
dont_have_features_set=set_our_features-smile_features
print(dont_have_features_set)

dont_have_features_list=list(dont_have_features_set)
print(len(dont_have_features_list))

{'pcm_fftMag_mfcc_sma_de[9]_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'audSpec_Rfilt_sma_de[21]_amean', 'pcm_fftMag_mfcc_sma_de[10]_stddev', 'audspecRasta_lengthL1norm_sma_de_amean', 'audSpec_Rfilt_sma_de[24]_amean', 'pcm_fftMag_mfcc_sma_de[1]_amean', 'pcm_fftMag_mfcc_sma[2]_amean', 'pcm_fftMag_mfcc_sma_de[8]_amean', 'pcm_fftMag_mfcc_sma[10]_stddev', 'pcm_fftMag_mfcc_sma_de[14]_stddev', 'audSpec_Rfilt_sma_de[7]_amean', 'pcm_fftMag_mfcc_sma[1]_stddev', 'pcm_fftMag_spectralHarmonicity_sma_de_amean', 'pcm_fftMag_spectralFlux_sma_de_amean', 'pcm_fftMag_spectralKurtosis_sma_de_amean', 'pcm_fftMag_mfcc_sma_de[4]_stddev', 'pcm_fftMag_mfcc_sma_de[12]_stddev', 'pcm_fftMag_fband250-650_sma_de_amean', 'pcm_fftMag_mfcc_sma[14]_stddev', 'pcm_fftMag_mfcc_sma[7]_stddev', 'audspec_lengthL1norm_sma_de_amean', 'audSpec_Rfilt_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma_de[5]_amean', 'audSpec_Rfilt_sma_de[8]_amean', 'pcm_fftMag_mfcc_sma[5]_amean', 'pcm_fftMag_spectralSlope_sma_de_amean', 'pcm_fftMag_spectral

In [6]:
smile_features_have_in_the_dataset=set_our_features - dont_have_features_set
print(smile_features_have_in_the_dataset)

smile_features_have_in_the_dataset=list(smile_features_have_in_the_dataset)
print(len(smile_features_have_in_the_dataset))

{'audSpec_Rfilt_sma_de[16]_stddev', 'pcm_fftMag_spectralRollOff90.0_sma_de_stddev', 'audSpec_Rfilt_sma[10]_amean', 'audSpec_Rfilt_sma[25]_amean', 'audSpec_Rfilt_sma_de[15]_stddev', 'audSpec_Rfilt_sma[17]_stddev', 'pcm_fftMag_spectralSlope_sma_amean', 'shimmerLocal_sma_stddev', 'audSpec_Rfilt_sma_de[23]_stddev', 'F0final_sma_amean', 'audSpec_Rfilt_sma[24]_stddev', 'audSpec_Rfilt_sma[19]_amean', 'jitterLocal_sma_de_stddev', 'shimmerLocal_sma_de_amean', 'audSpec_Rfilt_sma_de[7]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[17]_amean', 'audSpec_Rfilt_sma_de[10]_stddev', 'audSpec_Rfilt_sma[12]_stddev', 'audSpec_Rfilt_sma[3]_stddev', 'pcm_fftMag_spectralSkewness_sma_amean', 'jitterDDP_sma_stddev', 'pcm_RMSenergy_sma_stddev', 'audSpec_Rfilt_sma_de[6]_stddev', 'shimmerLocal_sma_de_stddev', 'audspecRasta_lengthL1norm_sma_stddev', 'audSpec_Rfilt_sma[6]_stddev', 'pcm_fftMag_spectralFlux_sma_stddev', 'audSpec_Rfilt_sma_de[2]_stddev', 'audSpec_Rfilt_sma[6]_amean', 'audSpec_Rfilt_sma_de[

In [7]:
drop_list=[]
#drop_list=dont_have_features_list
for i in range(0,len(dont_have_features_list)):
  drop_list.append(dont_have_features_list[i])

In [8]:
drop_list.append('Arousal_Value')
drop_list.append('Valence_Value')
drop_list.append('frameTime')

print(drop_list)
print(len(drop_list))

['pcm_fftMag_mfcc_sma_de[9]_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'audSpec_Rfilt_sma_de[21]_amean', 'pcm_fftMag_mfcc_sma_de[10]_stddev', 'audspecRasta_lengthL1norm_sma_de_amean', 'audSpec_Rfilt_sma_de[24]_amean', 'pcm_fftMag_mfcc_sma_de[1]_amean', 'pcm_fftMag_mfcc_sma[2]_amean', 'pcm_fftMag_mfcc_sma_de[8]_amean', 'pcm_fftMag_mfcc_sma[10]_stddev', 'pcm_fftMag_mfcc_sma_de[14]_stddev', 'audSpec_Rfilt_sma_de[7]_amean', 'pcm_fftMag_mfcc_sma[1]_stddev', 'pcm_fftMag_spectralHarmonicity_sma_de_amean', 'pcm_fftMag_spectralFlux_sma_de_amean', 'pcm_fftMag_spectralKurtosis_sma_de_amean', 'pcm_fftMag_mfcc_sma_de[4]_stddev', 'pcm_fftMag_mfcc_sma_de[12]_stddev', 'pcm_fftMag_fband250-650_sma_de_amean', 'pcm_fftMag_mfcc_sma[14]_stddev', 'pcm_fftMag_mfcc_sma[7]_stddev', 'audspec_lengthL1norm_sma_de_amean', 'audSpec_Rfilt_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma_de[5]_amean', 'audSpec_Rfilt_sma_de[8]_amean', 'pcm_fftMag_mfcc_sma[5]_amean', 'pcm_fftMag_spectralSlope_sma_de_amean', 'pcm_fftMag_spectral

In [9]:
print(len(feature_files))

1802


In [10]:

# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(drop_list, axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_v = []
mae_v = []
rmse_v = []
r2_v = []
f1_v = []
accuracy_v = []
precision_v = []
recall_v = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_val = X_reshaped[train_index], X_reshaped[test_index]
    y_v_train, y_v_val = y_v[train_index], y_v[test_index]

    # Split the validation set further into val and test sets
    val_size = len(X_val) // 2
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_v_val, y_v_test = y_v_val[:val_size], y_v_val[val_size:]

    # Build the BiLSTM model for Valence
    model_v = Sequential()
    model_v.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_v.add(Flatten())

    # Add a DNN layer after the BiLSTM
    model_v.add(Dense(512, activation='relu'))

    model_v.add(Dense(1))
    model_v.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_v.fit(X_train, y_v_train, validation_data=(X_val, y_v_val), epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_v = model_v.predict(X_test)

    # Reshape predictions_a to match the shape of y_a_test
    predictions_v = predictions_v.reshape(y_v_test.shape)

    # Calculate additional evaluation metrics
    mse_v.append(mean_squared_error(y_v_test, predictions_v))
    mae_v.append(mean_absolute_error(y_v_test, predictions_v))
    rmse_v.append(np.sqrt(mean_squared_error(y_v_test, predictions_v)))
    r2_v.append(r2_score(y_v_test, predictions_v))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_v = (predictions_v >= threshold).astype(int)
    y_v_test_binary = (y_v_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_v.append(f1_score(y_v_test_binary, binary_predictions_v))
    accuracy_v.append(accuracy_score(y_v_test_binary, binary_predictions_v))
    precision_v.append(precision_score(y_v_test_binary, binary_predictions_v))
    recall_v.append(recall_score(y_v_test_binary, binary_predictions_v))

# Calculate average performance across all folds
average_mse_v = np.mean(mse_v)
average_mae_v = np.mean(mae_v)
average_rmse_v = np.mean(rmse_v)
average_r2_v = np.mean(r2_v)
average_f1_v = np.mean(f1_v)
average_accuracy_v = np.mean(accuracy_v)
average_precision_v = np.mean(precision_v)
average_recall_v = np.mean(recall_v)

print(f'Average Valence MSE: {average_mse_v:.4f}')
print(f'Average Valence MAE: {average_mae_v:.4f}')
print(f'Average Valence RMSE: {average_rmse_v:.4f}')
print(f'Average Valence R2 Score: {average_r2_v:.4f}')
print(f'Average Valence F1 Score: {average_f1_v:.4f}')
print(f'Average Valence Accuracy: {average_accuracy_v:.4f}')
print(f'Average Valence Precision: {average_precision_v:.4f}')
print(f'Average Valence Recall: {average_recall_v:.4f}')

Epoch 1/25
2654/2654 [==============================] - 48s 15ms/step - loss: 0.0431 - val_loss: 0.0444
Epoch 2/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0386 - val_loss: 0.0405
Epoch 3/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0374 - val_loss: 0.0412
Epoch 4/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0367 - val_loss: 0.0407
Epoch 5/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0361 - val_loss: 0.0393
Epoch 6/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0356 - val_loss: 0.0386
Epoch 7/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0352 - val_loss: 0.0380
Epoch 8/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0348 - val_loss: 0.0379
Epoch 9/25
2654/2654 [==============================] - 38s 14ms/step - loss: 0.0344 - val_loss: 0.0394
Epoch 10/25
2654/2654 [==============================] - 38s 14m

In [2]:
! pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168842 sha256=e4484e9e51bc8f984f2aa03153452fbf87cfae0dfac54c8f36eba39fcc16e98d
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639


In [3]:
import opensmile
import numpy as np
import librosa

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
    #config_file_path="path_to_your_config_file.conf"
)

# Load the audio signal (replace with your own loading mechanism)
audio_path = '/content/drive/MyDrive/FYP_dataset/CNN Features/abc.wav'
#signal, sr= smile.process_file('/content/drive/MyDrive/FYP_dataset/CNN Features/abc.wav')

signal, sr = librosa.load(audio_path, sr=None)

# Segment the audio signal into 0.5-second segments
segment_duration = 0.5
samples_per_segment = int(segment_duration * sr)
timestamps = np.arange(0, len(signal) / sr, segment_duration)
segments = [signal[i:i + samples_per_segment] for i in range(0, len(signal), samples_per_segment)]

# Initialize a list to store features
features = []

# Extract features and timestamps for each segment
for timestamp, segment in zip(timestamps, segments):
    feature_dict = smile.process_signal(segment, sr)
    feature_dict['timestamp'] = timestamp  # Add the timestamp to the feature dictionary
    features.append(feature_dict)

<ipython-input-3-ba54e3932b98>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(audio_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: ignored